### Extraction of opinions

Imports

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import os

Utils

In [2]:
# define a function to rerurn a given child selector(s) from within a given ancestor
def extract(ancestor, selector=None, attribute = None, multiple = False):
    if selector:
        if multiple:
            if attribute:
                return [tag[attribute].strip for tag in ancestor.select(selector)]
            return [tag.text.strip for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()

In [3]:
selectors = {
    'opinion_id': (None, "data-entry-id"),
    'author': ("span.user-post__author-name",),
    'recommendation': ("span.user-post__author-recomendation > em",),
    'score': ("span.user-post__score-count",),
    'content': ("div.user-post__text",),
    'pros': ("review-feature__item--positive", None, True),
    'cons': ("review-feature__item--negative", None, True),
    'thumbs_up': ("button.vote-yes", "data-total-vote",),
    'thumbs_down': ("button.vote-no", "data-total-vote",),
    'date_published': ("span.user-post__published > time:nth-child(1)", "datetime"),
    'date_purchased': ("span.user-post__published > time:nth-child(2)", "datetime")
}

Obtaining the webpage

In [4]:
with open("./cookie.json", "r", encoding="UTF-8") as json_file:
    headers = json.load(json_file)
requests_url = "https://www.ceneo.pl/"
product_id = "161123001"
url = f"{requests_url}{product_id}#tab=reviews"
response = requests.get(url, headers = headers)

Extracting the opinions

In [5]:
all_opinions = []
while url is not None:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        page_doc = BeautifulSoup(response.text, 'html.parser')
        opinions = page_doc.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *values) # unpacks a tuple or list --> converts list of elements into seperate elements
                for key, values in selectors.items()
            }
            all_opinions.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_doc.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None

Generating json file

In [6]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as json_file:
    json.dump(all_opinions, json_file, indent=4, ensure_ascii=False)